<a href="https://colab.research.google.com/github/obinna23/dating-market-design/blob/main/Nahian_Haque%2C_Obinna_Ekeagwu_MS%26E_230_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

def gale_shapley(men_pref, women_pref, men_cutoff, women_cutoff):
    """
    Gale-Shapley algorithm for stable matching with cutoffs.

    Each man only considers women for which his actual preference exceeds men_cutoff,
    and each woman only accepts proposals from men for which her actual preference exceeds women_cutoff.

    Parameters:
      - men_pref: (n_men x n_women) array of men's actual preferences.
      - women_pref: (n_women x n_men) array of women's actual preferences.
      - men_cutoff: cutoff value for men (only consider women with men_pref > men_cutoff).
      - women_cutoff: cutoff value for women.

    Returns:
      - matches: list of (man_index, woman_index) tuples representing the final engagements.
      - men_decisions: (n_men x n_women) array recording men's proposals.
          * A 1 indicates that the man proposed (i.e. liked the candidate, since his preference exceeded men_cutoff).
          * A NaN indicates he never proposed to that candidate.
      - women_decisions: (n_women x n_men) array recording women's responses.
          * A 1 indicates that the woman accepted the proposal.
          * A 0 indicates that she rejected the proposal.
          * A NaN indicates no proposal was received from that man.
    """
    n_men, n_women = men_pref.shape
    # Initialize decision matrices with NaN (meaning "not yet swiped/proposed")
    men_decisions = np.full((n_men, n_women), np.nan)
    women_decisions = np.full((n_women, n_men), np.nan)

    # For each man, build a list of women indices he considers acceptable,
    # sorted in descending order of his preference.
    men_order = []
    for i in range(n_men):
        acceptable = np.where(men_pref[i, :] > men_cutoff)[0]
        # sort in descending order (highest preference first)
        sorted_idx = acceptable[np.argsort(-men_pref[i, acceptable])]
        men_order.append(list(sorted_idx))

    # Each man will propose in turn to the women in his list.
    # We'll track which proposal index he is on.
    proposal_index = [0] * n_men
    # All men start free.
    free_men = list(range(n_men))

    # Dictionary to record current engagements:
    # key: woman index, value: man index to whom she is currently engaged.
    engaged_to = {}

    # While there is a free man who still has someone to propose to:
    while free_men:
        i = free_men.pop(0)  # get the next free man

        # If he has proposed to all acceptable women, skip him.
        if proposal_index[i] >= len(men_order[i]):
            continue

        # The next woman in his sorted acceptable list:
        j = men_order[i][proposal_index[i]]
        proposal_index[i] += 1
        # Record his decision: he swiped right (i.e. likes) because his score is above cutoff.
        men_decisions[i, j] = 1

        # Woman j will only consider i if she finds him acceptable.
        if women_pref[j, i] <= women_cutoff:
            # She rejects immediately.
            women_decisions[j, i] = 0
            # Man i remains free if he has more women on his list.
            if proposal_index[i] < len(men_order[i]):
                free_men.append(i)
            continue

        # If woman j finds man i acceptable, check her current situation.
        if j not in engaged_to:
            # If she is free, she accepts.
            engaged_to[j] = i
            women_decisions[j, i] = 1
        else:
            # She is currently engaged; decide whether to keep her current partner.
            current_man = engaged_to[j]
            # Compare her preference scores.
            if women_pref[j, i] > women_pref[j, current_man]:
                # She prefers the new proposal.
                engaged_to[j] = i
                women_decisions[j, i] = 1
                # Mark that she is rejecting her previous partner.
                women_decisions[j, current_man] = 0
                # The rejected man becomes free again (if he has remaining options).
                if proposal_index[current_man] < len(men_order[current_man]):
                    free_men.append(current_man)
            else:
                # She rejects the new proposal.
                women_decisions[j, i] = 0
                # Man i remains free if he has more women to propose to.
                if proposal_index[i] < len(men_order[i]):
                    free_men.append(i)

    # Build the final list of matches from the engagements.
    matches = [(engaged_to[j], j) for j in engaged_to]
    return matches, men_decisions, women_decisions


In [ ]:
import numpy as np

def baseline(
    men_pref, women_pref,
    expected_men_pref, expected_women_pref,
    men_cutoff, women_cutoff,
    num_shown
):
    """
    Vectorized simulation for swipe dating app.

    Parameters:
      - men_pref: (n_men x n_women) array of men's actual preferences.
      - women_pref: (n_women x n_men) array of women's actual preferences.
      - expected_men_pref: (n_men x n_women) array of men's expected preferences.
      - expected_women_pref: (n_women x n_men) array of women's expected preferences.
      - men_cutoff: cutoff value for men (if actual score > cutoff, they like).
      - women_cutoff: cutoff value for women.
      - num_shown: number of candidates each user is shown per round (highest expected matches among those not yet seen).

    Returns:
      - matches: list of (man_index, woman_index) tuples where both users liked each other.
      - men_decisions: matrix (n_men x n_women) of men decisions (1 for like, 0 for reject, NaN for not yet swiped).
      - women_decisions: matrix (n_women x n_men) of women decisions.
    """
    n_men, n_women = men_pref.shape

    # Precompute sorted candidate lists (in descending order of expected preference)
    # For each man, sort women indices according to expected_men_pref.
    sorted_women = np.argsort(-expected_men_pref, axis=1)  # shape (n_men, n_women)
    # For each woman, sort men indices according to expected_women_pref.
    sorted_men = np.argsort(-expected_women_pref, axis=1)    # shape (n_women, n_men)

    # Pointers for each person into their sorted list (i.e. how many candidates they've seen)
    men_ptr = np.zeros(n_men, dtype=int)
    women_ptr = np.zeros(n_women, dtype=int)

    # Initialize decisions matrices (NaN indicates "not yet swiped").
    men_decisions = np.full((n_men, n_women), np.nan)
    women_decisions = np.full((n_women, n_men), np.nan)

    matches = set()

    # Men swipe round:
    for i in range(n_men):
        # If man i has already swiped on everyone, skip.
        if men_ptr[i] >= n_women:
            continue

        # Determine candidates for this round: next num_shown from sorted order.
        end = min(men_ptr[i] + num_shown, n_women)
        candidates = sorted_women[i, men_ptr[i]:end]
        # Compute swipe decisions based on actual preference.
        decisions = (men_pref[i, candidates] > men_cutoff).astype(int)
        men_decisions[i, candidates] = decisions
        men_ptr[i] = end  # advance pointer
        if candidates.size > 0:
            new_decision_made = True

    # Women swipe round:
    for j in range(n_women):
        if women_ptr[j] >= n_men:
            continue

        end = min(women_ptr[j] + num_shown, n_men)
        candidates = sorted_men[j, women_ptr[j]:end]
        decisions = (women_pref[j, candidates] > women_cutoff).astype(int)
        women_decisions[j, candidates] = decisions
        women_ptr[j] = end
        if candidates.size > 0:
            new_decision_made = True

    # Check for matches: We only need to consider pairs where both have swiped right.
    # Find indices where men swiped right.
    m_idx, w_idx = np.where(men_decisions == 1)
    for i, j in zip(m_idx, w_idx):
        # A match exists if woman j also swiped right on man i.
        if women_decisions[j, i] == 1:
            matches.add((i, j))

    return list(matches), men_decisions, women_decisions

In [ ]:
# Albers & Ledwig 2019
stop_params = {
  1: { "r": 1, "c": 0.3678, "comp_ratio": 0.3678},
  2: { "r": 1, "c": 0.2545, "comp_ratio": 0.4119},
  3: { "r": 2, "c": 0.3475, "comp_ratio": 0.4449},
  4: { "r": 2, "c": 0.2928, "comp_ratio": 0.4785},
  5: { "r": 2, "c": 0.2525, "comp_ratio": 0.4999},
  6: { "r": 2, "c": 0.2217, "comp_ratio": 0.5148},
  7: { "r": 3, "c": 0.28, "comp_ratio": 0.5308},
  8: { "r": 3, "c": 0.2549, "comp_ratio": 0.5453},
  9: { "r": 3, "c": 0.2338, "comp_ratio": 0.5567},
  10: { "r": 3, "c": 0.2159, "comp_ratio": 0.566},
  11: { "r": 4, "c": 0.257, "comp_ratio": 0.574},
  12: { "r": 4, "c": 0.241, "comp_ratio": 0.5834},
  13: { "r": 4, "c": 0.2267, "comp_ratio": 0.5914},
  14: { "r": 4, "c": 0.214, "comp_ratio": 0.5983},
  15: { "r": 4, "c": 0.2026, "comp_ratio": 0.6043},
  16: { "r": 4, "c": 0.1924, "comp_ratio": 0.6096},
  17: { "r": 5, "c": 0.2231, "comp_ratio": 0.6155},
  18: { "r": 5, "c": 0.2133, "comp_ratio": 0.6211},
  19: { "r": 5, "c": 0.2042, "comp_ratio": 0.6261},
  20: { "r": 5, "c": 0.1959, "comp_ratio": 0.6306},
  21: { "r": 5, "c": 0.1882, "comp_ratio": 0.6347},
  22: { "r": 5, "c": 0.1811, "comp_ratio": 0.6384},
  23: { "r": 6, "c": 0.2054, "comp_ratio": 0.6426},
  24: { "r": 6, "c": 0.1985, "comp_ratio": 0.6465},
  25: { "r": 6, "c": 0.1919, "comp_ratio": 0.6502},
  26: { "r": 6, "c": 0.1858, "comp_ratio": 0.6535},
  27: { "r": 6, "c": 0.18, "comp_ratio": 0.6566},
  28: { "r": 6, "c": 0.1746, "comp_ratio": 0.6595},
  29: { "r": 7, "c": 0.1947, "comp_ratio": 0.6625},
  30: { "r": 7, "c": 0.1893, "comp_ratio": 0.6655},
  31: { "r": 7, "c": 0.1842, "comp_ratio": 0.6684},
  32: { "r": 7, "c": 0.1793, "comp_ratio": 0.6711},
  33: { "r": 7, "c": 0.1747, "comp_ratio": 0.6736},
  34: { "r": 7, "c": 0.1703, "comp_ratio": 0.676},
  35: { "r": 7, "c": 0.1662, "comp_ratio": 0.6782},
  36: { "r": 8, "c": 0.183, "comp_ratio": 0.6805},
  37: { "r": 8, "c": 0.1788, "comp_ratio": 0.6829},
  38: { "r": 8, "c": 0.1748, "comp_ratio": 0.6851},
  39: { "r": 8, "c": 0.171, "comp_ratio": 0.6873},
  40: { "r": 8, "c": 0.1673, "comp_ratio": 0.6893},
  41: { "r": 8, "c": 0.1638, "comp_ratio": 0.6912},
  42: { "r": 8, "c": 0.1605, "comp_ratio": 0.693},
  43: { "r": 9, "c": 0.175, "comp_ratio": 0.6948},
  44: { "r": 9, "c": 0.1716, "comp_ratio": 0.6968},
  45: { "r": 9, "c": 0.1683, "comp_ratio": 0.6986},
  46: { "r": 9, "c": 0.1651, "comp_ratio": 0.7004},
  47: { "r": 9, "c": 0.1621, "comp_ratio": 0.7021},
  48: { "r": 9, "c": 0.1592, "comp_ratio": 0.7037},
  49: { "r": 9, "c": 0.1563, "comp_ratio": 0.7052},
  50: { "r": 9, "c": 0.1536, "comp_ratio": 0.7067}
}

In [ ]:
import numpy as np

def like_limit(
    men_pref, women_pref,
    expected_men_pref, expected_women_pref,
    men_cutoff, women_cutoff,
    num_shown,
    k
):
    """
    Vectorized simulation for swipe dating app using a modified selection rule.

    Each agent (men and women) proceeds as follows in each round:
      1. They view the next `num_shown` candidates in order of descending expected preference.
      2. They automatically reject (swipe left on) the top c*num_shown candidates, where c is
         given by stop_params[k]["c"].
      3. They then consider the remaining candidates and select (swipe right on) the next k
         candidates (by expected preference order) whose true preference exceeds the threshold.
         The threshold is defined as the rth highest true preference among the rejected
         c*num_shown candidates, with r given by stop_params[k]["r"].
      4. All candidates in the current block that are not selected are treated as rejected.

    Parameters:
      - men_pref: (n_men x n_women) array of men's true preferences.
      - women_pref: (n_women x n_men) array of women's true preferences.
      - expected_men_pref: (n_men x n_women) array of men's expected preferences.
      - expected_women_pref: (n_women x n_men) array of women's expected preferences.
      - num_shown: number of candidates each user is shown per round.
      - k: number of candidates to possibly like (swipe right on) in the second step.
      - stop_params: dictionary where the key is k and the value is a dictionary with keys "c" and "r".
                    For example, stop_params[k] might be {"c": 0.2, "r": 1}.

    Returns:
      - matches: list of (man_index, woman_index) tuples where both users liked each other.
      - men_decisions: matrix (n_men x n_women) of men's decisions (1 for like, 0 for reject, NaN for not yet swiped).
      - women_decisions: matrix (n_women x n_men) of women's decisions.
    """
    n_men, n_women = men_pref.shape

    # Precompute sorted candidate lists (in descending order of expected preference)
    # For each man, sort women indices according to expected_men_pref.
    sorted_women = np.argsort(-expected_men_pref, axis=1)  # shape (n_men, n_women)
    # For each woman, sort men indices according to expected_women_pref.
    sorted_men = np.argsort(-expected_women_pref, axis=1)    # shape (n_women, n_men)

    # Pointers for each person into their sorted list (i.e. how many candidates they've seen)
    men_ptr = np.zeros(n_men, dtype=int)
    women_ptr = np.zeros(n_women, dtype=int)

    # Initialize decisions matrices (NaN indicates "not yet swiped").
    men_decisions = np.full((n_men, n_women), np.nan)
    women_decisions = np.full((n_women, n_men), np.nan)

    matches = set()

    # Extract the parameters for this value of k.
    c = stop_params[k]["c"]
    r = stop_params[k]["r"]

    # Men swipe round:
    for i in range(n_men):
        if men_ptr[i] >= n_women:
            continue

        end = min(men_ptr[i] + num_shown, n_women)
        candidates = sorted_women[i, men_ptr[i]:end]
        np.random.shuffle(candidates)
        if candidates.size == 0:
            continue

        # Determine how many candidates to automatically reject.
        num_reject = int(c * num_shown)
        num_reject = min(num_reject, len(candidates))
        # Automatically reject these candidates.
        rejected_candidates = candidates[:num_reject]

        # Compute the threshold based on the rth highest true preference among the rejected.
        if len(rejected_candidates) >= r:
            # Get true preferences of the rejected candidates.
            rejected_true = men_pref[i, rejected_candidates]
            # Sort in descending order.
            sorted_rejected_true = np.sort(rejected_true)[::-1]
            threshold = sorted_rejected_true[r - 1]
        else:
            threshold = -np.inf
        threshold = max(threshold, men_cutoff)

        # Now, from the remaining candidates, select the next k that have true preference > threshold.
        selected_candidates = []
        for candidate in candidates[num_reject:]:
            if men_pref[i, candidate] > threshold:
                selected_candidates.append(candidate)
                if len(selected_candidates) >= k:
                    break

        # Assign decisions for this block: selected candidates get a 1, all others get a 0.
        for candidate in candidates:
            decision = 1 if candidate in selected_candidates else 0
            men_decisions[i, candidate] = decision

        men_ptr[i] = end
        new_decision_made = True

    # Women swipe round:
    for j in range(n_women):
        if women_ptr[j] >= n_men:
            continue

        end = min(women_ptr[j] + num_shown, n_men)
        candidates = sorted_men[j, women_ptr[j]:end]
        np.random.shuffle(candidates)
        if candidates.size == 0:
            continue

        num_reject = int(c * num_shown)
        num_reject = min(num_reject, len(candidates))
        rejected_candidates = candidates[:num_reject]

        if len(rejected_candidates) >= r:
            rejected_true = women_pref[j, rejected_candidates]
            sorted_rejected_true = np.sort(rejected_true)[::-1]
            threshold = sorted_rejected_true[r - 1]
        else:
            threshold = -np.inf
        threshold = max(threshold, women_cutoff)

        selected_candidates = []
        for candidate in candidates[num_reject:]:
            if women_pref[j, candidate] > threshold:
                selected_candidates.append(candidate)
                if len(selected_candidates) >= k:
                    break

        for candidate in candidates:
            decision = 1 if candidate in selected_candidates else 0
            women_decisions[j, candidate] = decision

        women_ptr[j] = end

    # Check for matches: A match exists if both the man and woman swiped right.
    m_idx, w_idx = np.where(men_decisions == 1)
    for i, j in zip(m_idx, w_idx):
        if women_decisions[j, i] == 1:
            matches.add((i, j))


    return list(matches), men_decisions, women_decisions


In [ ]:
import numpy as np

def hide_top(
    men_pref, women_pref,
    expected_men_pref, expected_women_pref,
    men_cutoff, women_cutoff,
    num_shown,
    p  # p is a percentage indicating the top portion to skip
):
    """
    Vectorized simulation for swipe dating app with a twist:
    each agent sees the top num_shown candidates from the bottom (100 - p)% of their expected preferences.

    Parameters:
      - men_pref: (n_men x n_women) array of men's actual preferences.
      - women_pref: (n_women x n_men) array of women's actual preferences.
      - expected_men_pref: (n_men x n_women) array of men's expected preferences.
      - expected_women_pref: (n_women x n_men) array of women's expected preferences.
      - men_cutoff: cutoff value for men (if actual score > cutoff, they like).
      - women_cutoff: cutoff value for women.
      - num_shown: number of candidates each user is shown per round.
      - p: percentage of top expected candidates to ignore (i.e. only show candidates from the bottom (100-p)%)

    Returns:
      - matches: list of (man_index, woman_index) tuples where both users liked each other.
      - men_decisions: matrix (n_men x n_women) of men's decisions.
      - women_decisions: matrix (n_women x n_men) of women's decisions.
    """
    n_men, n_women = men_pref.shape

    # Precompute sorted candidate lists (in descending order of expected preference)
    # For each man, sort women indices according to expected_men_pref.
    sorted_women = np.argsort(-expected_men_pref, axis=1)  # shape (n_men, n_women)
    # For each woman, sort men indices according to expected_women_pref.
    sorted_men = np.argsort(-expected_women_pref, axis=1)    # shape (n_women, n_men)

    # Pointers for each person into their sorted list (i.e. how many candidates they've seen)
    men_ptr = np.zeros(n_men, dtype=int)
    women_ptr = np.zeros(n_women, dtype=int)

    # Initialize decisions matrices (NaN indicates "not yet swiped").
    men_decisions = np.full((n_men, n_women), np.nan)
    women_decisions = np.full((n_women, n_men), np.nan)

    matches = set()

    # Determine how many candidates to skip (i.e., the top p% are ignored)
    skip_men = int(p/100 * n_women)   # For each man, skip the top p% women.
    skip_women = int(p/100 * n_men)     # For each woman, skip the top p% men.

    # Replaces globally defined men_ptr, women_ptr varialbles
    # Initialize pointers to start at the beginning of the bottom (100-p)% of the candidate list.
    men_ptr = np.full(n_men, skip_men, dtype=int)
    women_ptr = np.full(n_women, skip_women, dtype=int)

    matches = set()
    new_decision_made = True
    round_num = 1

    while new_decision_made:
        new_decision_made = False

        # Men swipe round:
        for i in range(n_men):
            # If man i has already swiped on everyone in the bottom (100-p)% set, skip.
            if men_ptr[i] >= n_women:
                continue

            # Determine candidates for this round: next num_shown from the restricted list.
            end = min(men_ptr[i] + num_shown, n_women)
            candidates = sorted_women[i, men_ptr[i]:end]
            # Compute swipe decisions based on actual preference.
            decisions = (men_pref[i, candidates] > men_cutoff).astype(int)
            men_decisions[i, candidates] = decisions
            men_ptr[i] = end  # advance pointer
            if candidates.size > 0:
                new_decision_made = True

        # Women swipe round:
        for j in range(n_women):
            if women_ptr[j] >= n_men:
                continue

            end = min(women_ptr[j] + num_shown, n_men)
            candidates = sorted_men[j, women_ptr[j]:end]
            decisions = (women_pref[j, candidates] > women_cutoff).astype(int)
            women_decisions[j, candidates] = decisions
            women_ptr[j] = end
            if candidates.size > 0:
                new_decision_made = True

        # Check for matches: consider pairs where both have swiped right.
        m_idx, w_idx = np.where(men_decisions == 1)
        for i, j in zip(m_idx, w_idx):
            # A match exists if woman j also swiped right on man i.
            if women_decisions[j, i] == 1:
                matches.add((i, j))

        round_num += 1

    return list(matches), men_decisions, women_decisions


In [ ]:
def super_like(
    men_pref, women_pref,
    expected_men_pref, expected_women_pref,
    men_cutoff, women_cutoff,
    num_shown,
    b, m
):
    """
    Vectorized simulation for a swipe dating app with superlikes.

    Each agent (man or woman) is given m superlikes. They do not have the option to
    superlike for their first (num_shown * c) candidates (where c = stop_params[m]['c']).
    After that, they will superlike the next m candidates for whom their true preference
    is higher than the r-th highest true preference among those first candidates (with
    r = stop_params[m]['r']).

    When an agent superlikes a candidate, the candidate's true preference for that agent
    increases by b (but never exceeds 1).

    Parameters:
      - men_pref: (n_men x n_women) array of men's actual preferences.
      - women_pref: (n_women x n_men) array of women's actual preferences.
      - expected_men_pref: (n_men x n_women) array of men's expected preferences.
      - expected_women_pref: (n_women x n_men) array of women's expected preferences.
      - men_cutoff: cutoff value for men (if actual score > cutoff, they like).
      - women_cutoff: cutoff value for women.
      - num_shown: number of candidates each user is shown per round.
      - b: boost amount added when a candidate is superliked.
      - m: number of superlikes available to each agent.
      - stop_params: a dict keyed by m that gives a dictionary with keys 'c' and 'r',
                     where c determines the number of candidates (num_shown*c) in the initial
                     phase and r the rank used for computing the superlike threshold.

    Returns:
      - matches: list of (man_index, woman_index) tuples where both users liked (or superliked) each other.
      - men_decisions: matrix (n_men x n_women) of men decisions
                        (0 for reject, 1 for normal like, 2 for superlike, NaN for not yet swiped).
      - women_decisions: matrix (n_women x n_men) of women decisions.
    """
    n_men, n_women = men_pref.shape

    # Precompute sorted candidate lists (in descending order of expected preference)
    # For each man, sort women indices according to expected_men_pref.
    sorted_women = np.argsort(-expected_men_pref, axis=1)  # shape (n_men, n_women)
    # For each woman, sort men indices according to expected_women_pref.
    sorted_men = np.argsort(-expected_women_pref, axis=1)    # shape (n_women, n_men)

    # Pointers for each person into their sorted list (i.e. how many candidates they've seen)
    men_ptr = np.zeros(n_men, dtype=int)
    women_ptr = np.zeros(n_women, dtype=int)

    # Initialize decisions matrices (NaN indicates "not yet swiped").
    men_decisions = np.full((n_men, n_women), np.nan)
    women_decisions = np.full((n_women, n_men), np.nan)

    matches = set()

    # Determine the number of candidates in the initial (no superlike) phase.
    phase_threshold = int(num_shown * stop_params[m]['c'])
    r_val = stop_params[m]['r']  # rank (1-indexed)

    # For each agent, keep track of:
    # - the list of true preference values from the first phase,
    # - the computed threshold (None until computed),
    # - and the number of superlikes remaining.
    men_first_phase = [[] for _ in range(n_men)]
    men_threshold   = [None] * n_men
    men_superlikes_remaining = np.full(n_men, m)

    women_first_phase = [[] for _ in range(n_women)]
    women_threshold   = [None] * n_women
    women_superlikes_remaining = np.full(n_women, m)

    # ---- Men swipe round ----
    for i in range(n_men):
        if men_ptr[i] >= n_women:
            continue

        end = min(men_ptr[i] + num_shown, n_women)
        candidates = sorted_women[i, men_ptr[i]:end]
        np.random.shuffle(candidates)
        # Process each candidate in this round individually.
        for j_local, candidate in enumerate(candidates):
            overall_index = men_ptr[i] + j_local

            # Check if we are still in the initial (no superlike) phase.
            if overall_index < phase_threshold:
                # Normal decision based on true preference.
                decision = 1 if men_pref[i, candidate] > men_cutoff else 0
                # Save this true preference value to later compute the superlike threshold.
                men_first_phase[i].append(men_pref[i, candidate])
            else:
                # Once we have reached phase_threshold, compute the threshold if not yet computed.
                if men_threshold[i] is None and len(men_first_phase[i]) >= phase_threshold:
                    sorted_vals = sorted(men_first_phase[i], reverse=True)
                    # r_val-th highest (assuming r_val is at least 1)
                    men_threshold[i] = sorted_vals[r_val - 1]
                    men_threshold[i] = max(men_threshold[i], men_cutoff)
                # Decide whether to superlike.
                if (men_superlikes_remaining[i] > 0 and
                    men_pref[i, candidate] > men_threshold[i]):
                    decision = 2  # superlike marker
                    men_superlikes_remaining[i] -= 1
                    # When a man superlikes a woman, boost that woman's true preference for him.
                    women_pref[candidate, i] = min(1, women_pref[candidate, i] + b)
                else:
                    # Otherwise, use the normal decision rule.
                    decision = 1 if men_pref[i, candidate] > men_cutoff else 0

            men_decisions[i, candidate] = decision
        men_ptr[i] = end
        if candidates.size > 0:
            new_decision_made = True

    # ---- Women swipe round ----
    for j in range(n_women):
        if women_ptr[j] >= n_men:
            continue

        end = min(women_ptr[j] + num_shown, n_men)
        candidates = sorted_men[j, women_ptr[j]:end]
        np.random.shuffle(candidates)
        for k_local, candidate in enumerate(candidates):
            overall_index = women_ptr[j] + k_local

            if overall_index < phase_threshold:
                decision = 1 if women_pref[j, candidate] > women_cutoff else 0
                women_first_phase[j].append(women_pref[j, candidate])
            else:
                if women_threshold[j] is None and len(women_first_phase[j]) >= phase_threshold:
                    sorted_vals = sorted(women_first_phase[j], reverse=True)
                    women_threshold[j] = sorted_vals[r_val - 1]
                    women_threshold[j] = max(women_threshold[j], women_cutoff)
                if (women_superlikes_remaining[j] > 0 and
                    women_pref[j, candidate] > women_threshold[j]):
                    decision = 2  # superlike
                    women_superlikes_remaining[j] -= 1
                    # When a woman superlikes a man, boost that man's true preference for her.
                    men_pref[candidate, j] = min(1, men_pref[candidate, j] + b)
                else:
                    decision = 1 if women_pref[j, candidate] > women_cutoff else 0

            women_decisions[j, candidate] = decision
        women_ptr[j] = end

    # ---- Check for matches ----
    # We treat both a normal like (1) and a superlike (2) as a "like" for matching.
    # Since not-yet-swiped entries are NaN, we convert them to 0 for this check.
    men_like = np.nan_to_num(men_decisions, nan=0) >= 1
    # Note: women_decisions is (n_women x n_men) so we use its transpose.
    women_like = np.nan_to_num(women_decisions.T, nan=0) >= 1
    match_mask = men_like & women_like
    m_idx, w_idx = np.where(match_mask)
    for i, j in zip(m_idx, w_idx):
        matches.add((i, j))

    return list(matches), men_decisions, women_decisions


In [ ]:
"""
import numpy as np

# Set the number of men and women
n_men = 100
n_women = 80


# Generate uniform scores for men and women
men_scores = np.random.uniform(0, 1, n_men)
women_scores = np.random.uniform(0, 1, n_women)

# Create preference matrices:
# For women_pref, we generate a (n_women x n_men) matrix of random values,
# then add 15% of the men_scores replicated across each row.
women_pref = 0.85 * np.random.uniform(0, 1, size=(n_women, n_men)) + \
            0.15 * np.tile(men_scores, (n_women, 1))

# For men_pref, we generate a (n_men x n_women) matrix of random values,
# then add 15% of the women_scores replicated across each row.
men_pref = 0.85 * np.random.uniform(0, 1, size=(n_men, n_women)) + \
          0.15 * np.tile(women_scores, (n_men, 1))

# For expected preferences, we mix the above preferences with new random matrices.
# Note: replicate(n_men, runif(n_women, 0, 1)) produces a (n_women x n_men) matrix.
expected_women_pref = 0.5 * women_pref + \
                      0.5 * np.random.uniform(0, 1, size=(n_women, n_men))

# replicate(n_women, runif(n_men, 0, 1)) produces a (n_men x n_women) matrix.
expected_men_pref = 0.5 * men_pref + \
                    0.5 * np.random.uniform(0, 1, size=(n_men, n_women))

# Set cutoff values
men_cutoff = 0.4
women_cutoff = 0.6

# Set num_shown: number of candidates shown per round
num_shown = 30
"""

'\nimport numpy as np\n\n# Set the number of men and women\nn_men = 100\nn_women = 80\n\n\n# Generate uniform scores for men and women\nmen_scores = np.random.uniform(0, 1, n_men)\nwomen_scores = np.random.uniform(0, 1, n_women)\n\n# Create preference matrices:\n# For women_pref, we generate a (n_women x n_men) matrix of random values,\n# then add 15% of the men_scores replicated across each row.\nwomen_pref = 0.85 * np.random.uniform(0, 1, size=(n_women, n_men)) +             0.15 * np.tile(men_scores, (n_women, 1))\n\n# For men_pref, we generate a (n_men x n_women) matrix of random values,\n# then add 15% of the women_scores replicated across each row.\nmen_pref = 0.85 * np.random.uniform(0, 1, size=(n_men, n_women)) +           0.15 * np.tile(women_scores, (n_men, 1))\n\n# For expected preferences, we mix the above preferences with new random matrices.\n# Note: replicate(n_men, runif(n_women, 0, 1)) produces a (n_women x n_men) matrix.\nexpected_women_pref = 0.5 * women_pref +      

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def simulate_match(n_men, n_women, function, iters, param_1, param_2):
    # Lists to store simulation results
    avg_man_true_preferences = []    # average true preference score for men's matches per simulation
    avg_woman_true_preferences = []  # average true preference score for women's matches per simulation
    num_matches_distribution = []    # total number of matches per simulation
    num_men_matched_distribution = []  # number of men who got matched per simulation
    num_women_matched_distribution = []  # number of women who got matched per simulation
    num_unstable_pairs_distribution = [] # number of unstable pairs per simulation

    men_cutoff = 0.4
    women_cutoff = 0.6

    for sim in range(iters):
        # Generate expected preferences (used by the matching algorithm)
        men_scores = np.random.uniform(0, 1, n_men)
        women_scores = np.random.uniform(0, 1, n_women)

        # Create preference matrices:
        # For women_pref, we generate a (n_women x n_men) matrix of random values,
        # then add 15% of the men_scores replicated across each row.
        women_pref = 0.85 * np.random.uniform(0, 1, size=(n_women, n_men)) + \
                     0.15 * np.tile(men_scores, (n_women, 1))

        # For men_pref, we generate a (n_men x n_women) matrix of random values,
        # then add 15% of the women_scores replicated across each row.
        men_pref = 0.85 * np.random.uniform(0, 1, size=(n_men, n_women)) + \
                   0.15 * np.tile(women_scores, (n_men, 1))

        # For expected preferences, we mix the above preferences with new random matrices.
        expected_women_pref = 0.5 * women_pref + \
                              0.5 * np.random.uniform(0, 1, size=(n_women, n_men))
        expected_men_pref = 0.5 * men_pref + \
                            0.5 * np.random.uniform(0, 1, size=(n_men, n_women))

        # Run matching
        num_shown = 15
        matches, men_decisions, women_decisions = function(
            men_pref, women_pref,
            expected_men_pref, expected_women_pref,
            men_cutoff, women_cutoff, num_shown,
            param_1, param_2
        )

        # Build dictionaries for quick lookup of each agent's match.
        man_to_woman = {m: j for m, j in matches}
        woman_to_man = {j: m for m, j in matches}

        # --- 1. Compute average true preference scores for matched pairs ---
        # For men: look up true preference scores from men_pref for each man's match.
        man_scores_list = []
        for i in range(n_men):
            if i in man_to_woman:
                j = man_to_woman[i]
                man_scores_list.append(men_pref[i, j])
        avg_man = np.mean(man_scores_list) if len(man_scores_list) > 0 else np.nan
        avg_man_true_preferences.append(avg_man)

        # For women: similarly, look up true preference scores from women_pref.
        woman_scores_list = []
        for j in range(n_women):
            if j in woman_to_man:
                i = woman_to_man[j]
                woman_scores_list.append(women_pref[j, i])
        avg_woman = np.mean(woman_scores_list) if len(woman_scores_list) > 0 else np.nan
        avg_woman_true_preferences.append(avg_woman)

        # --- 2. Count total matches and matched agents ---
        total_matches = len(matches)
        num_matches_distribution.append(total_matches)
        num_men_matched_distribution.append(len(man_to_woman))
        num_women_matched_distribution.append(len(woman_to_man))

        # --- 3. Count unstable pairs ---
        # We only consider pairs where both agents are matched.
        unstable_count = 0
        for m in man_to_woman:
            current_woman = man_to_woman[m]
            # Check all other matched women (excluding his current partner)
            for w in woman_to_man:
                if w == current_woman:
                    continue
                # If m prefers w over his current match...
                if men_pref[m, w] > men_pref[m, current_woman]:
                    current_man_for_w = woman_to_man[w]
                    # ...and if w prefers m over her current match, then (m, w) is an unstable pair.
                    if women_pref[w, m] > women_pref[w, current_man_for_w]:
                        unstable_count += 1
        num_unstable_pairs_distribution.append(unstable_count)
        if (total_matches < unstable_count):
          ValueError("There cannot be more unstable matches than matches.")

    return (avg_man_true_preferences,
            avg_woman_true_preferences,
            num_matches_distribution,
            num_men_matched_distribution,
            num_women_matched_distribution,
            num_unstable_pairs_distribution)

In [ ]:
def gale_wrapper(men_pref, women_pref,
            expected_men_pref, expected_women_pref,
            men_cutoff, women_cutoff, num_shown,
            param_1, param_2):
  return gale_shapley(
            expected_men_pref, expected_women_pref,
            men_cutoff, women_cutoff
        )

def base_wrapper(men_pref, women_pref,
            expected_men_pref, expected_women_pref,
            men_cutoff, women_cutoff, num_shown,
            param_1, param_2):
  return baseline(men_pref, women_pref,
            expected_men_pref, expected_women_pref,
            men_cutoff, women_cutoff, num_shown)

def limit_wrapper(men_pref, women_pref,
            expected_men_pref, expected_women_pref,
            men_cutoff, women_cutoff, num_shown,
            param_1, param_2):
  return like_limit(men_pref, women_pref,
            expected_men_pref, expected_women_pref,
            men_cutoff, women_cutoff, num_shown,
            param_1)

def hide_wrapper(men_pref, women_pref,
            expected_men_pref, expected_women_pref,
            men_cutoff, women_cutoff, num_shown,
            param_1, param_2):
  return hide_top(men_pref, women_pref,
            expected_men_pref, expected_women_pref,
            men_cutoff, women_cutoff, num_shown,
            param_1)

# super_like does not neet wrapper

In [ ]:
gale_results = simulate_match(100, 80, gale_wrapper, 500, None, None)

base_results = simulate_match(100, 80, base_wrapper, 500, None, None)

In [ ]:
# After the simulations, you can analyze or plot the distributions.

# --- Plot histograms of the results ---

plt.figure()
plt.hist(gale_results[0], bins=20, alpha = 0.5)
plt.hist(base_results[0], bins=20, alpha = 0.5)
plt.suptitle("Distribution of Average True Preference for Men's Matches", fontsize = 12)
plt.title("Imperfect Information Gale-Shapley vs Baseline Swiping Model, #men = 100, #women = 80", fontsize = 6)
plt.xlabel("Average True Preference")
plt.ylabel("Frequency")
plt.legend(["Gale-Shapley", "Baseline"])
plt.show()

plt.figure()
plt.hist(gale_results[1], bins=20, alpha = 0.5)
plt.hist(base_results[1], bins=20, alpha = 0.5)
plt.suptitle("Distribution of Average True Preference for Women's Matches", fontsize = 12)
plt.title("Imperfect Information Gale-Shapley vs Baseline Swiping Model, #men = 100, #women = 80", fontsize = 6)
plt.xlabel("Average True Preference")
plt.ylabel("Frequency")
plt.legend(["Gale-Shapley", "Baseline"])
plt.show()

plt.figure()
plt.hist(gale_results[4], bins=20, alpha = 0.5)
plt.hist(base_results[4], bins=20, alpha = 0.5)
plt.suptitle("Distribution of Women Matched", fontsize = 12)
plt.title("Imperfect Information Gale-Shapley vs Baseline Swiping Model, #men = 100, #women = 80", fontsize = 6)
plt.xlabel("Average True Preference")
plt.ylabel("Frequency")
plt.legend(["Gale-Shapley", "Baseline"])
plt.show()

plt.figure()
plt.hist(gale_results[5], bins=20, alpha = 0.5)
plt.hist(base_results[5], bins=20, alpha = 0.5)
plt.suptitle("Distribution of Unstable Pairs", fontsize = 12)
plt.title("Imperfect Information Gale-Shapley vs Baseline Swiping Model, #men = 100, #women = 80", fontsize = 6)
plt.xlabel("Unstable Pairs")
plt.ylabel("Frequency")
plt.legend(["Gale-Shapley", "Baseline"])
plt.show()

In [ ]:
import scipy.stats as sci
print(sci.ttest_ind(gale_results[0], base_results[0]))
print(sci.ttest_ind(gale_results[1], base_results[1]))
print(sci.ttest_ind(gale_results[2], base_results[2]))
print(sci.ttest_ind(gale_results[3], base_results[3]))
print(sci.ttest_ind(gale_results[4], base_results[4]))
print(sci.ttest_ind(gale_results[5], base_results[5]))

In [ ]:
limit_mens_rate = []
limit_womens_rate = []
limit_matches = []
limit_men_matched = []
limit_women_matched = []
limit_unstable = []

for k in [2, 5, 10, 20, 29]:
  limit_results = simulate_match(400, 320, limit_wrapper, 100, k, None)
  limit_mens_rate.append(np.mean(limit_results[0]))
  limit_womens_rate.append(np.mean(limit_results[1]))
  limit_matches.append(np.mean(limit_results[2]))
  limit_men_matched.append(np.mean(limit_results[3]))
  limit_women_matched.append(np.mean(limit_results[4]))
  limit_unstable.append(np.mean(limit_results[5]))

In [ ]:
# Create the plot
plt.plot([2, 5, 10, 20, 29], limit_mens_rate, linestyle='dotted', marker='o', label='Men', color = "blue")
plt.plot([2, 5, 10, 20, 29], limit_womens_rate, linestyle='dotted', marker='s', label='Women', color = "pink")

# Labels and title
plt.xlabel("Like Limit (k)")
plt.ylabel("Average Preference for Match")
plt.title("Like Limit Algorithm, Average Preference for Match")
plt.legend()

# Show the plot
plt.show()


In [ ]:
hide_mens_rate = []
hide_womens_rate = []
hide_matches = []
hide_men_matched = []
hide_women_matched = []
hide_unstable = []

for p in [0.1, 1, 10, 25, 50, 75, 99]:
  hide_results = simulate_match(400, 320, hide_wrapper, 500, p, None)
  hide_mens_rate.append(np.mean(hide_results[0]))
  hide_womens_rate.append(np.mean(hide_results[1]))
  hide_matches.append(np.mean(hide_results[2]))
  hide_men_matched.append(np.mean(hide_results[3]))
  hide_women_matched.append(np    .mean(hide_results[4]))
  hide_unstable.append(np.mean(hide_results[5]))

In [ ]:
hide_matches

In [ ]:
hide_unstable

In [ ]:
# Create the plot
plt.plot([0.1, 1, 10, 25, 50, 75, 99], hide_mens_rate, linestyle='dotted', marker='o', label='Men', color = "blue")
plt.plot([0.1, 1, 10, 25, 50, 75, 99], hide_womens_rate, linestyle='dotted', marker='s', label='Women', color = "pink")

# Labels and title
plt.xlabel("top % hidden (p)")
plt.ylabel("Average Preference for Match")
plt.title("Hide Top Algorithm, Average Preference for Match")
plt.legend()

# Show the plot
plt.show()


In [ ]:
# Create the plot
plt.plot([0.1, 1, 10, 25, 50, 75, 99], hide_men_matched, linestyle='dotted', marker='o', label='Men', color = "blue")
plt.plot([0.1, 1, 10, 25, 50, 75, 99], hide_women_matched, linestyle='dotted', marker='s', label='Women', color = "pink")

# Labels and title
plt.xlabel("top % hidden (p)")
plt.ylabel("#")
plt.title("Hide Top Algorithm, Number of people with 1+ match")
plt.legend()

# Show the plot
plt.show()


In [ ]:
# Create the plot
plt.plot([0.1, 1, 10, 25, 50, 75, 99], hide_matches, linestyle='dotted', marker='o', label='All', color = "green")
plt.plot([0.1, 1, 10, 25, 50, 75, 99], hide_unstable, linestyle='dotted', marker='s', label='Unstable', color = "red")

# Labels and title
plt.xlabel("Top % excluded (p)")
plt.ylabel("#")
plt.title("Hide Top Algorithm, Number of (Unstable) Matches")
plt.legend()

# Show the plot
plt.show()


In [ ]:
# Create the plot
plt.plot([2, 5, 10, 20, 29], limit_matches, linestyle='dotted', marker='o', label='All', color = "green")
plt.plot([2, 5, 10, 20, 29], limit_unstable, linestyle='dotted', marker='s', label='Unstable', color = "red")

# Labels and title
plt.xlabel("Like Limit (k)")
plt.ylabel("#")
plt.title("Like Limit Algorithm, Number of (Unstable) Matches")
plt.legend()

# Show the plot
plt.show()


Key metrics (for presentation):
* Number of matches
* Number of people matched
  * Histogram of matches by people
  * Gini coefficient
* Average preference of matches
* Number of unstable pairs
  * (both would leave all patners)





In [ ]:
# Create the plot
plt.plot([0.01, 0.1, 0.25, 0.5], limit_mens_rate, linestyle='dotted', marker='o', label='Men', color = "blue")
plt.plot([0.01, 0.1, 0.25, 0.5],, limit_womens_rate, linestyle='dotted', marker='s', label='Women', color = "pink")

# Labels and title
plt.xlabel("Like Limit (k)")
plt.ylabel("Average Preference for Match")
plt.title("Like Limit Algorithm, Average Preference for Match")
plt.legend()

# Show the plot
plt.show()
